# Exploring Newest Trends in Most Populous Cities of the United States

### Introduction

Company X desires to know which venues are trending in the various major US cities. This knowledge will help them solve a larger problem: in which companies should Company X invest in. Company X is particularly interested in whether or not there are correlations between percent-increase/percent-decrease of the most populous cities with the types of venues that are newly available (venues opening within the past decade).

##### Data Collection

To start with, I will scrape Wikipedia's page for the list of most populous US cities with their corresponding percent-increase/percent-decrease:

https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

Then, using this initial data set with the corresponding GPS locations (included in the data table), I will obtain venues using the Foursquare API and investigate the trending venues.

I perform the scrape below:

In [1]:
# import necessary libraries

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population' # define desired url
results = requests.get(url) # access url location
soup = BeautifulSoup(results.text,'html.parser') # parse through html of url and store page info
print(soup.prettify()) # print html content from webpage

In [3]:
city_pop_table = soup.find('table',{'class': 'wikitable sortable'}) # find the first table of class 'wikitable sortable'
city_pop_head = city_pop_table.tr.find_all('th') # create html table headers list

column_names = [] # create empty list

for header in city_pop_head:
    column_names.append(header.text.strip().replace(',','_')) # collects column names as list
print(column_names)

['2018rank', 'City', 'State[c]', '2018estimate', '2010Census', 'Change', '2016 land area', '2016 population density', 'Location']


In [4]:
file_name = 'us_cities_data.csv' # give name to file

f = open(file_name,'w+', encoding='utf-8') # utf-8 encoding is used in html;
# will need to clean up

for i in range(len(column_names)): # write to file all column names as header to file
    if column_names[i] != column_names[-1]:
        f.write(column_names[i] + ',')
    else:
        f.write(column_names[i] + '\n')

table_rows = city_pop_table.find_all('tr') # list of all tr

for row in table_rows: # grab data from each cell and write to file
    cells = row.find_all('td')
    if len(cells) > 1:
        for i, cell in enumerate(cells):
            cell_data = cell.text.strip().replace(',','')
            if cells[i] != cells[-1]:
                if i == 7 or i == 9: # ignore metric units
                # assuming this is an American company so English units are desired
                    continue
                f.write(cell_data + ',')
            else:
                f.write(cell_data + '\n')
                    
f.close() # be sure to close the file!!!

In [5]:
us_cities_df = pd.read_csv('us_cities_data.csv') # read csv file to obtain dataframe
us_cities_df.head() # take a peak at dataframe

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Location
0,1,New York[d],New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


### Data Cleaning

In [6]:
cities_cleaned = [] # initialize an empty list

for i, text in enumerate(us_cities_df.index): # cleans city names and appends to list
    cities_cleaned.append(us_cities_df.loc[i,'City'].split('[')[0])
    
print(cities_cleaned) # take a peak

['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'Fort Worth', 'Columbus', 'San Francisco', 'Charlotte', 'Indianapolis', 'Seattle', 'Denver', 'Washington', 'Boston', 'El Paso', 'Detroit', 'Nashville', 'Portland', 'Memphis', 'Oklahoma City', 'Las Vegas', 'Louisville', 'Baltimore', 'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Mesa', 'Sacramento', 'Atlanta', 'Kansas City', 'Colorado Springs', 'Miami', 'Raleigh', 'Omaha', 'Long Beach', 'Virginia Beach', 'Oakland', 'Minneapolis', 'Tulsa', 'Arlington', 'Tampa', 'New Orleans', 'Wichita', 'Cleveland', 'Bakersfield', 'Aurora', 'Anaheim', 'Honolulu', 'Santa Ana', 'Riverside', 'Corpus Christi', 'Lexington', 'Stockton', 'Henderson', 'Saint Paul', 'St. Louis', 'Cincinnati', 'Pittsburgh', 'Greensboro', 'Anchorage', 'Plano', 'Lincoln', 'Orlando', 'Irvine', 'Newark', 'Toledo', 'Durham', 'Chula Vista', 'Fort Wayne', 'Jersey City', 'St. Petersburg

In [7]:
us_cities_df['City'] = pd.Series(cities_cleaned) # repleace 'City' column with newly cleaned names
us_cities_df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Location
0,1,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [8]:
location_cleaned = [] # initialize empty list

for i, content in enumerate(us_cities_df.index): # cleaning location lat/long coords
    location_cleaned.append(us_cities_df.loc[i,'Location'].replace('\ufeff','').split('/')[2].split('(')[0].replace(' ',''))
    
lat_coords = [] # initialize
long_coords = [] # initialize

for i, text in enumerate(location_cleaned): # separate lat and long coords into separate lists
    lat, long = location_cleaned[i].split(';')
    lat_coords.append(lat)
    long_coords.append(long)

us_cities_df['Lat'] = pd.Series(lat_coords) # add column for latidude
us_cities_df['Long'] = pd.Series(long_coords) # add column for longitude
us_cities_df.head() # take a peak

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Location,Lat,Long
0,1,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,33.5722,-112.0901


In [9]:
us_cities_df.drop('Location',axis=1,inplace=True) # drop messy location column
us_cities_df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Lat,Long
0,1,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33.5722,-112.0901


In [10]:
us_cities_df.rename(columns={'2018ranks': '2018 Rank',
                             'State[c]': 'State',
                             '2018estimate': '2018 Estimate',
                             '2010Census': '2010 Census',
                             '2016 land area': 'Land Area (sq mi)',
                             '2016 population density': 'Population Density (per sq mi)'},
                    inplace=True) # renaming columns for cleaner look and easier use
us_cities_df.head()

,2018rank,City,State,2018 Estimate,2010 Census,Change,Land Area (sq mi),Population Density (per sq mi),Lat,Long
0,1,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33.5722,-112.0901


In [11]:
us_cities_df.drop('2018rank',axis=1,inplace=True) # rank unnecessary with pandas' indexing
us_cities_df.head()

,City,State,2018 Estimate,2010 Census,Change,Land Area (sq mi),Population Density (per sq mi),Lat,Long
0,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,28317/sq mi,40.6635,-73.9387
1,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,8484/sq mi,34.0194,-118.4108
2,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,11900/sq mi,41.8376,-87.6818
3,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,3613/sq mi,29.7866,-95.3909
4,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,3120/sq mi,33.5722,-112.0901


In [12]:
land_area = [] # initialize
pop_density = [] # initialize

for i, text in enumerate(us_cities_df.index): # clean land area and population density columns
    land_area.append(us_cities_df.loc[i,'Land Area (sq mi)'].replace('\xa0sq\xa0mi',''))
    pop_density.append(us_cities_df.loc[i,'Population Density (per sq mi)'].split('/')[0])

us_cities_df['Land Area (sq mi)'] = pd.Series(land_area)
us_cities_df['Population Density (per sq mi)'] = pd.Series(pop_density)
us_cities_df.head() # take a peak

,City,State,2018 Estimate,2010 Census,Change,Land Area (sq mi),Population Density (per sq mi),Lat,Long
0,New York,New York,8398748,8175133,+2.74%,301.5,28317,40.6635,-73.9387
1,Los Angeles,California,3990456,3792621,+5.22%,468.7,8484,34.0194,-118.4108
2,Chicago,Illinois,2705994,2695598,+0.39%,227.3,11900,41.8376,-87.6818
3,Houston,Texas,2325502,2100263,+10.72%,637.5,3613,29.7866,-95.3909
4,Phoenix,Arizona,1660272,1445632,+14.85%,517.6,3120,33.5722,-112.0901


The data needs further cleaning but this current dataframe should serve as a good first place to start for this project.